# Statistics Canada Analysis

## Summary

1-2 sentences description of analysis

## Introduction

Intro 

- Background info relating to the topic 
- Narrowing down of variables will be justified here
- Will use 3 references: Definitions of certain term(s), background info or inspiration for the study, narrowing down of variables due to prior research (hopefully from a peer reviewed article).

### Research Question

- Explicitly stated research question

#### Dataset Description

(Based on Overview and Scope of Coverage Sections)
- cites the pdf provided with the dataset

#### Description of Relevent Variables

(Based on "Variables Description" provided in the PDF)

- cites the pdf provided with the dataset

## Methods and Results

### Description of Methods

- 
-
-
-
-
-
-
-

### Preliminary Analysis

#### Loading the Data

In [41]:
# Importing the appropriate packages

import pandas as pd
import sklearn as sk
# import os

In [40]:
# Loading the data

data = pd.read_csv('data\\CIS-72M0003-E-2017-Annual\\CIS-72M0003-E-2017-Annual_F1.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92292 entries, 0 to 92291
Columns: 194 entries, YEAR to VERDATE
dtypes: float64(6), int64(185), object(3)
memory usage: 136.6+ MB


#### Cleaning and Wrangling

#### Visualisations

### Training the Model

### Testing

## Discussion of Results

## References